# The Theory

## Capital Asset Pricing Model
The motivation behind Sharpe's (1964) CAPM model is closely related to the Efficient Market Hypothesis (EMH) introduced by Fama (1965). More specifically, Markowitz (1952) established the concept of the efficient frontier, which states that the only way to increase returns is by taking on more risk. Sharpe built on this idea by explaining excess returns (over the risk-free rate) in relation to the underlying risk. If risk fully explains excess returns, it supports the conclusion that markets are efficient, and the only way to achieve higher returns is by moving along the efficient frontier through increased risk-taking. 

For this to be true, the following are insufficient, but necessary: 
- The model explain well the excess return (High $R^2$);
- The error terms has a mean of 0 ($E[\epsilon] =0$).

### Model
$
R_i - R_f = \beta_i (R_m - R_f) + \epsilon_i
$

Where:  
- $ R_i - R_f  $: Asset's Return Excess (The return of an individual asset (e.g., a stock) over the risk-free rate.)
- $ R_i $: Return on asset $ i $  
- $ R_f $: Risk-free rate  
- $ R_m $: Return on the market  
- $ \beta_i $: Sensitivity of Asset $ i $ to market $ m $  
- $ \epsilon_i $:  idiosyncratic risk of asset $i$

This can alternatively be read as:

The excess return = alpha + market risk + error terms.


## The Fama-French Factor Model
The issue with CAPM lies in its simplicity and reliance on a single factor — market risk $(R_m - R_f)$ — to explain asset returns. If one accepts the efficient frontier, it raises the question: why would an investor achieve higher returns for the same level of risk? Yet, numerous papers have shown evidence of such anomalies (CITE PAPERS). This has led some to argue that CAPM's failure to fully explain market returns suggests that markets may not be efficient. However, this interpretation is flawed due to the joint hypothesis problem. Testing market efficiency using CAPM conflates three possibilities: (1) the model is correct, and the market is inefficient, (2) the model is incorrect, but the market is efficient, or (3) both the model and the market hypothesis are wrong. This ambiguity complicates conclusions about market efficiency and highlights the challenges of using a model to test the Efficient Market Hypothesis. While this may seem unscientific — given science is often defined by falsifiability — it remains a core issue in financial theory.

In response, researchers identified additional factors, driven by anomalies that CAPM could not explain. Two key anomalies persisted: the size effect—where smaller firms tend to outperform larger ones for a given level of risk—and the value effect, where high book-to-market firms outperform low book-to-market firms. These persistent deviations from CAPM predictions led Fama and French to develop multi-factor models, starting with the 3-Factor Model, which incorporates size (SMB) and value (HML) factors. They later expanded it to the 5-Factor Model, adding profitability (RMW) and investment (CMA) to better capture variations in return Both models led to higher explanatory power and were robust at the time of their publication (see discussion at the end). .

### 3-Factor Model

$
R_i - R_f = \beta_i (R_m - R_f) + \gamma_i SMB + \delta_i HML + \epsilon_i
$

Wher  
- $ \beta_i, \gamma_i, \delta_i $: Sensitivities to the facto
- $ R_m - R_f $: Excess market return  s  
- $ SMB $: Small-minus-big factor (size)  
- $ HML $: High-minus-low factor (value)

### 5-Factor Model
$
R_i - R_f = \beta_i (R_m - R_f) + \gamma_i SMB + \delta_i HML + \theta_i RMW + \rho_i CMA + \epsilon_i
$

Where:
- $ \beta_i, \gamma_i, \delta_i, \theta_i, \rho_i $: Sensitivities to the factors
- $ R_m - R_f $: Excess market return    
- $ SMB $: Small-minus-big factor (size)  
- $ HML $: High-minus-low factor (value)
- $ RMW $: Robust-minus-weak factor (profitability- )
- $ CMA $: Conservative-minus-aggressive factor (investment)
  
More information regarding how factor are built here: https://www.tidy-finance.org/python/replicating-fama-and-french-factors.html

# Empirical Estimation

In [6]:
#!pip install -r "requirements.txt"
# https://github.com/tidy-finance/website/blob/main/requirements.txt

In [7]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import sqlite3  # For working with SQLite databases
import statsmodels.formula.api as smf  # For regression analysis using formulas

from regtabletotext import prettify_result  # For formatting regression tables into text
from statsmodels.regression.rolling import RollingOLS  # For rolling regression models
from plotnine import *  # For ggplot2-style data visualization
from mizani.breaks import date_breaks  # For customizing date breaks in plots
from mizani.formatters import percent_format, date_format  # For formatting plot labels
from joblib import Parallel, delayed, cpu_count  # For parallel processing
from itertools import product  # For generating Cartesian product of input iterables


We start by downloading Fama-French factors (e.g., Fama and French 1993) and portfolio returns commonly used in empirical asset pricing. Fortunately, the pandas-datareader package provides a simple interface to read data from Kenneth French’s Data Library. Ii is also possible to download it from here (Fama/French 5 Factors (2x3)) https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html 

We also import our WRDS datas, see Loading Financial Datas.ipynb. for more informations.

In [9]:
tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")
# See Loading Financial Datas.ipynb. If you do not have access to WRDS, see WRDS Dummy Datas.ipynb. 
crsp_monthly = (pd.read_sql_query(
    sql=("SELECT permno, gvkey, date, industry, ret_excess, mktcap, "
         "mktcap_lag, exchange FROM crsp_monthly"),
    con=tidy_finance,
    parse_dates={"date"})
  .dropna()
)

factors_ff3_monthly = pd.read_sql_query(
  sql="SELECT date, mkt_excess FROM factors_ff3_monthly",
  con=tidy_finance,
  parse_dates={"date"}
)

factors_ff3_monthly = pd.read_sql_query(
  sql="SELECT date, mkt_excess, smb, hml FROM factors_ff3_monthly",
  con=tidy_finance,
  parse_dates={"date"}
)

factors_ff5_monthly = pd.read_sql_query(
  sql="SELECT date, mkt_excess, smb, hml, rmw, cma FROM factors_ff5_monthly",
  con=tidy_finance,
  parse_dates={"date"}
)

crsp_monthly_5 = (crsp_monthly
  .merge(factors_ff5_monthly, how="left", on="date")
)

In [10]:
#crsp_monthly_3.head(),crsp_monthly_5.head()

In [11]:
model_beta = (smf.ols(
    formula="ret_excess ~ mkt_excess",
    data=crsp_monthly_5.query("permno == 14593")) # This is Apple.
  .fit()
)
prettify_result(model_beta)

OLS Model:
ret_excess ~ mkt_excess

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.010       0.005        2.046    0.041
mkt_excess     1.384       0.109       12.696    0.000

Summary statistics:
- Number of observations: 516
- R-squared: 0.239, Adjusted R-squared: 0.237
- F-statistic: 161.191 on 1 and 514 DF, p-value: 0.000



The results of the OLS regression analyzing Apple’s excess returns relative to the market provide a nuanced view of the stock’s performance under the Capital Asset Pricing Model (CAPM). With a statistically significant beta estimate of 1.384 (𝑝<0.001), the findings indicate that Apple exhibits higher systematic risk than the overall market. Specifically, the coefficient suggests that a 1% change in the market’s excess return is associated with a 1.384% change in Apple’s excess return, on average. 

In other words, one of the main drivers of Apple's performance is that it is riskier. It also has a slight Alpha (p < 0.05). That being said, this model only explains 24% of the excess return. 

(For portfolio managers, the implications are twofold: Apple represents a leveraged exposure to market dynamics, suitable for investors aiming to capitalize on upward market trends. At the same time, its ability to deliver alpha justifies its inclusion in portfolios where outperformance is sought, even in market-neutral or risk-adjusted strategies. Such strategies could be to long Apple and short S&P 500.)

Let's see of the CAMP and Fama-French Models perform in the manufacturing industry.

In [13]:
model_beta = (smf.ols(
    formula="ret_excess ~ mkt_excess",
    data=crsp_monthly_5.query("industry == 'Manufacturing'")) 
  .fit()
)
prettify_result(model_beta)

OLS Model:
ret_excess ~ mkt_excess

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.002       0.000       12.317      0.0
mkt_excess     1.197       0.003      349.251      0.0

Summary statistics:
- Number of observations: 1,267,846
- R-squared: 0.088, Adjusted R-squared: 0.088
- F-statistic: 121,976.359 on 1 and 1267844 DF, p-value: 0.000



### CAMP Model: Apple Vs. Manufacturing Industry
What we’ve just observed is that the beta of the industry is less volatile than Apple’s, which could justify a lower return as well. Both are still cyclical stocks, meaning they move in tandem with the market and are more sensitive to market-wide movements. Moreover, CAMP explains solely 9% of the whole market excess return. 

In [15]:
model_fama_3 = (smf.ols(
    formula="ret_excess ~ mkt_excess + smb + hml",
    data=crsp_monthly_5.query("industry == 'Manufacturing'")) 
  .fit()
)
model_fama_5 = (smf.ols(
    formula="ret_excess ~ mkt_excess + smb + hml + rmw + cma",
    data=crsp_monthly_5.query("industry == 'Manufacturing'")) 
  .fit()
)

In [16]:
prettify_result(model_fama_3),prettify_result(model_fama_5)

OLS Model:
ret_excess ~ mkt_excess + smb + hml

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.001       0.000        6.605    0.000
mkt_excess     1.030       0.004      284.914    0.000
smb            1.010       0.005      196.446    0.000
hml            0.009       0.005        1.662    0.096

Summary statistics:
- Number of observations: 1,267,846
- R-squared: 0.115, Adjusted R-squared: 0.115
- F-statistic: 54,759.940 on 3 and 1267842 DF, p-value: 0.000

OLS Model:
ret_excess ~ mkt_excess + smb + hml + rmw + cma

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.002       0.000       10.911     0.00
mkt_excess     1.023       0.004      266.201     0.00
smb            0.947       0.005      172.851     0.00
hml            0.008       0.007        1.126     0.26
rmw           -0.237       0.007      -32.167     0.00
cma            0.044       0.011        3.956     0.00

Summary statistics:
- Number of obser

(None, None)

### Comparison of CAMP, Factor 3 and Factor 5 in Manufacturing
#### CAMP Vs. Factor 3
Once we include the factors from the Fama-French 3-Models (and robust to 5-Factor Models), it is notable that the market excess decreases significantly (~20%). This indicates that less of the excess is attributed to market risk. The reduction could be due to the CAPM's omission of additional risk factors, which led to an upward bias in the market excess under the CAMP. Specifically, the Size (SMB) and, to a lesser extent, Value (HML) factors provide alternative explanations for excess returns. In fact, we see the market excess approaching 1, which is exactly what we would expect, as manufacturing is a diverse industry with players of various sizes.

Interestingly, despite these factors being positively correlated with market risk (as evidenced by the reduced market excess coefficient), they capture a distinct set of idiosyncratic risks associated with small-cap and value stocks that are not fully explained by market trends, thus, explaining the additional return relative to market movement. This diversification is particularly valuable for portfolio management, as it enhances the risk-return tradeoff by adding imperfectly correlated sources of return to the portfolio, thus reducing portfolio's drawdown.

#### Factor 3 Vs. Factor 5
First, it is worth emphasizing that the factor-3 model is particularly robust compared to the factor-5 model, with the factor-3 coefficients remaining largely unchanged (except for a slight decrease in the Size factor, SMB). It is clear that small-cap stocks significantly contribute to manufacturing returns. However, the negative coefficient for Profitability (RMW) is noteworthy, as it suggests that less profitable manufacturing firms tend to contribute more to the sector's returns. Although this may seem odd, it aligns with the Kenneth French Data Library but contrasts with a joint CFA Institute report, which highlights RMW as "the single factor that has consistently delivered excess returns over time".

Moreover, the Factor-5 model highlights that conservative investment strategies tend to yield higher returns.

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html 
https://blogs.cfainstitute.org/investor/2022/01/10/fama-and-french-the-five-factor-model-revisited/



In [18]:
model_fama_5_no_fe = (smf.ols(
    formula="ret_excess ~ mkt_excess + smb + hml + rmw + cma",  # No industry FEs
    data=crsp_monthly_5  # Entire dataset (including multiple industries)
).fit())
model_fama_3_no_fe = (smf.ols(
    formula="ret_excess ~ mkt_excess + smb + hml",  
    data=crsp_monthly_5  # Entire dataset (including multiple industries)
).fit())



In [19]:
prettify_result(model_fama_3_no_fe), prettify_result(model_fama_5_no_fe)

OLS Model:
ret_excess ~ mkt_excess + smb + hml

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.000       0.000        3.880      0.0
mkt_excess     0.989       0.002      424.121      0.0
smb            0.897       0.003      268.193      0.0
hml            0.080       0.003       23.777      0.0

Summary statistics:
- Number of observations: 3,116,737
- R-squared: 0.096, Adjusted R-squared: 0.096
- F-statistic: 110,831.106 on 3 and 3116733 DF, p-value: 0.000

OLS Model:
ret_excess ~ mkt_excess + smb + hml + rmw + cma

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.001       0.000       11.622      0.0
mkt_excess     0.972       0.002      391.968      0.0
smb            0.846       0.004      234.963      0.0
hml            0.133       0.005       28.828      0.0
rmw           -0.181       0.005      -38.526      0.0
cma           -0.074       0.007      -10.507      0.0

Summary statistics:
- Number of obse

(None, None)

### Factor 3 and Factor 5 For the Whole Market
The results show that the portfolio's excess returns are primarily driven by market risk (β ≈ 1), which is expected since it represents the entire market. The portfolio has a strong sensitivity to the size factor (SMB), indicating a preference for small-cap stocks. There is also a modest tilt toward value stocks (HML).

Additionally, the 5-factor model highlights negative sensitivity to profitability (RMW = -0.181) and investment (CMA = -0.074), suggesting a tilt toward less profitable firms and those with aggressive investment strategies.s.s.s.

### Is Fama-French factors outdated?
A key concern is that the Factor-3 model, which was a seminal paper in the 1990s, may no longer hold true in today's market. To address this, we briefly test it with data split into pre- and post-2000 periods. We find that the results are surprisingly robust. That being said, we do see a weakness in the impact of the Small and Value stock, but that could also be that these factors have been partly priced in the pre-2000, reducing the premium. 

We could indeed try a shorter time horizon, but, shorter periods (e.g., less than 20 years) can obscure the true long-term return trends. For example, the U.S. stock market rallied exceptionally well in the 1990s, but investors who went all-in then experienced the "lost decade" of poor returns during the 2000s. Worse, if they exited during this period, they would have missed the fantastic bull market that began in the early 2010s. 

One alternative is to follow Buffett's advice of investing 100% in the S&P 500. However, this may not be optimal, as there are uncorrelated risks available that can improve diversification. Additionally, while current market valuations reflect strong fundamentals, the implicit pricing assumption of two-digit growth in future profits is reminiscent of the dot-com bubble. The key difference this time is that today's companies tend to have far stronger fundamentals than those during the dot-com era.

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html https://blogs.cfainstitute.org/investor/2022/01/10/fama-and-french-the-five-factor-model-revisited/
ed/ 

In [22]:
# Create period dummy variables
crsp_monthly_5['Pre_2000'] = (crsp_monthly_5['date'].dt.year < 2000).astype(int)  # 1980-2000
crsp_monthly_5['Post_2000'] = (crsp_monthly_5['date'].dt.year >= 2000).astype(int)  # Post-2000

# Generate interaction terms for each factor
crsp_monthly_5['mkt_excess_1980_2000'] = crsp_monthly_5['mkt_excess'] * crsp_monthly_5['Pre_2000']
crsp_monthly_5['smb_1980_2000'] = crsp_monthly_5['smb'] * crsp_monthly_5['Pre_2000']
crsp_monthly_5['hml_1980_2000'] = crsp_monthly_5['hml'] * crsp_monthly_5['Pre_2000']
crsp_monthly_5['rmw_1980_2000'] = crsp_monthly_5['rmw'] * crsp_monthly_5['Pre_2000']
crsp_monthly_5['cma_1980_2000'] = crsp_monthly_5['cma'] * crsp_monthly_5['Pre_2000']

crsp_monthly_5['mkt_excess_post_2000'] = crsp_monthly_5['mkt_excess'] * crsp_monthly_5['Post_2000']
crsp_monthly_5['smb_post_2000'] = crsp_monthly_5['smb'] * crsp_monthly_5['Post_2000']
crsp_monthly_5['hml_post_2000'] = crsp_monthly_5['hml'] * crsp_monthly_5['Post_2000']
crsp_monthly_5['rmw_post_2000'] = crsp_monthly_5['rmw'] * crsp_monthly_5['Post_2000']
crsp_monthly_5['cma_post_2000'] = crsp_monthly_5['cma'] * crsp_monthly_5['Post_2000']


In [23]:
# Run the Fama-French 5-Factor model with period-specific factors
model_fama_5_split = smf.ols(
    formula=("ret_excess ~ mkt_excess_1980_2000 + smb_1980_2000 + hml_1980_2000 + "
             "rmw_1980_2000 + cma_1980_2000 + "
             "mkt_excess_post_2000 + smb_post_2000 + hml_post_2000 + "
             "rmw_post_2000 + cma_post_2000"),
    data=crsp_monthly_5
).fit()
model_fama_3_split = smf.ols(
    formula=("ret_excess ~ mkt_excess_1980_2000 + smb_1980_2000 + hml_1980_2000 + "
             "mkt_excess_post_2000 + smb_post_2000 + hml_post_2000"),
    data=crsp_monthly_5
).fit()

# Prettify and display results
prettify_result(model_fama_3_split), prettify_result(model_fama_5_split)


OLS Model:
ret_excess ~ mkt_excess_1980_2000 + smb_1980_2000 + hml_1980_2000 
 + mkt_excess_post_2000 + smb_post_2000 + hml_post_2000

Coefficients:
                      Estimate  Std. Error  t-Statistic  p-Value
Intercept                0.000       0.000        4.489    0.000
mkt_excess_1980_2000     0.956       0.003      298.129    0.000
smb_1980_2000            0.990       0.005      218.821    0.000
hml_1980_2000            0.164       0.005       30.955    0.000
mkt_excess_post_2000     1.060       0.004      300.636    0.000
smb_post_2000            0.781       0.005      157.427    0.000
hml_post_2000            0.004       0.004        1.011    0.312

Summary statistics:
- Number of observations: 3,116,737
- R-squared: 0.097, Adjusted R-squared: 0.097
- F-statistic: 55,778.909 on 6 and 3116730 DF, p-value: 0.000

OLS Model:
ret_excess ~ mkt_excess_1980_2000 + smb_1980_2000 + hml_1980_2000 
 + rmw_1980_2000 + cma_1980_2000 + mkt_excess_post_2000 + smb_post_2000 + hml_post_2000

(None, None)